In [ ]:
kubectl config use-context kubernetes-admin@kubernetes
$ENV:ACCEPT_EULA='yes'
$ENV:AZDATA_USERNAME='admin'
$ENV:AZDATA_PASSWORD='P@ssw0rd'
$Env:SPN_AUTHORITY='https://login.microsoftonline.com'
$AzureSub='<YourSubscription>'
$AzureRG="arcDemo"
$AzureLAWS="LogAnalyticsArcDemos"
$AzureSP='arcdemo-loganalytics'

In [ ]:
azdata arc dc create --connectivity-mode Indirect -n arc-dc-local -ns arc -s $AzureSub -g arcDemo -l eastus -sc local-storage --profile-name azure-arc-kubeadm

In [ ]:
kubectl get pods -n arc

Let's take a look at the Wizard - therefore, install Arc Extension

Let's also connect to our instance in ADS.

In [ ]:
azdata login -ns arc

In [ ]:
azdata arc sql mi list

In [ ]:
azdata arc sql mi create -n local-mi-01 -scl local-storage -scd local-storage -scdl local-storage

In [ ]:
# azdata arc sql mi delete -n local-mi-01

In [ ]:
azdata arc sql mi list

Connect to MI 1 in ADS

In [ ]:
kubectl exec local-mi-01-0 -n arc -c arc-sqlmi -- wget https://github.com/Microsoft/sql-server-samples/releases/download/adventureworks/AdventureWorks2019.bak -O /var/opt/mssql/data/AdventureWorks2019.bak

In [ ]:
kubectl exec local-mi-01-0 -n arc -c arc-sqlmi -- /opt/mssql-tools/bin/sqlcmd -S localhost -U admin -P "P@ssw0rd" -Q "RESTORE DATABASE [AdventureWorks2019] FROM  DISK = N'/var/opt/mssql/data/AdventureWorks2019.bak' WITH  FILE = 1,  MOVE N'AdventureWorks2017' TO N'/var/opt/mssql/data/AdventureWorks2019.mdf',  MOVE N'AdventureWorks2017_log' TO N'/var/opt/mssql/data/AdventureWorks2019_log.ldf',  NOUNLOAD,  STATS = 5"

We can also copy and restore:

In [ ]:
kubectl cp ..\..\Files\AdventureWorks2017.bak arc/local-mi-01-0:var/opt/mssql/data/AdventureWorks2017.bak -c arc-sqlmi

Check out Grafana

But how can I see that in Azure?

In [ ]:
az login

In [ ]:
az account set -s $AzureSub

In [ ]:
$SP=(az ad sp create-for-rbac --name http://$AzureSP)
$Env:SPN_CLIENT_ID=($SP | ConvertFrom-Json).appId
$Env:SPN_CLIENT_SECRET=($SP | ConvertFrom-Json).password
$Env:SPN_TENANT_ID=($SP | ConvertFrom-Json).tenant
$SP.Replace($Env:SPN_CLIENT_SECRET,'XXXXX')

In [ ]:
az role assignment create --assignee $Env:SPN_CLIENT_ID --role "Monitoring Metrics Publisher" --scope subscriptions/$AzureSub

In [ ]:
$LAWS=(az monitor log-analytics workspace create -g $AzureRG -n $AzureLAWS)
$Env:WORKSPACE_ID=($LAWS | ConvertFrom-Json).customerId

In [ ]:
$LAWSKEYS=(az monitor log-analytics workspace get-shared-keys -g $AzureRG -n $AzureLAWS)
$Env:WORKSPACE_SHARED_KEY=($LAWSKEYS | ConvertFrom-Json).primarySharedKey

Indirect vs. Direct Mode:

In [ ]:
azdata arc dc export -t metrics --path metrics.json --force
azdata arc dc export -t logs --path logs.json --force

In [ ]:
azdata arc dc upload --path metrics.json
azdata arc dc upload --path logs.json

In [ ]:
azdata arc sql mi delete -n local-mi-01
azdata arc dc delete -ns arc -n arc-dc-local --force

In [ ]:
azdata arc dc create --connectivity-mode Direct -n arc-dc-local-direct -ns arc -s $AzureSub -g arcDemo -l eastus -sc local-storage --profile-name azure-arc-kubeadm

In [ ]:
#az ad sp delete --id $Env:SPN_CLIENT_ID